# Financial Solution Accelerator: Drawing a Company Ecosystem Graph and Analyzing it with Embeddings
This accelerator will help you process Financial Annual Reports (10K filings) or even Wikipedia data about companies, using John Snow Labs Finance NLP **Named Entity Recognition, Relation Extraction and Assertion Status**, to extract the following information about companies:
- Information about the Company itself (`Trading Symbol`, `State`, `Address`, Contact Information) and other names the Company is known by (`alias`, `former name`).
- Other Companies mentioned in the report as `competitors`: we will also run a "Competitor check", to understand if another company is just in the ecosystem / supply chain of the company or it is really a competitor
- People (usually management and C-level) working in that company and their past experiences, including roles and companies
- `Acquisitions` events, including the acquisition dates. `Subsidiaries` mentioned.
- Temporality (`past`, `present`, `future`) and Certainty (`possible`) of events described, including `Forward-looking statements`.

Also, John Snow Labs provides with offline modules to check for Edgar database (**Entity Linking** to resolve an organization name to its official name and **Chunk Mappers** to map a normalized name to Edgar Database), which are quarterly updated. We will using them to retrieve the `official name of a company`, `former names`, `dates where names where changed`, etc.

The final aim of this accelerator is to help you analyze companies information...

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/im1.png" alt="drawing" width="600"/>

... create a graph...

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/img6.png" alt="drawing" width="400"/>

... and even being able to run Graph Embeddings on top of the graph you extract (for example, to infer new relations to green nodes given the grey ones in the picture);

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/im4.png" alt="drawing" width="400"/>

# Install Spark NLP

In [ ]:
!pip install johnsnowlabs

In [ ]:
from johnsnowlabs import *
jsl.install(json_license_path=[your_path_to_json_license])

# Starting a session

In [ ]:
jsl.start(json_license_path=[your_path_to_json_license])

# Imports

In [ ]:
import os
import sys
import time
import json
import functools 
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import spatial

### NetworkX and Plotly aux functions 
We will use [NetworkX](https://networkx.org/) to store the graph and [Plotly](https://plotly.com/) to visualize it.

In [ ]:
!pip install networkx plotly

In [ ]:
import networkx as nx
G = nx.Graph()

In [ ]:
import plotly.graph_objects as go
import random

def get_nodes_from_graph(graph, pos, node_color):
  node_x = []
  node_y = []
  texts = []
  hovers = []
  for node in graph.nodes():
    entity = graph.nodes[node]['attr_dict']['entity']
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    texts.append(node)
    hovers.append(entity)

  node_trace = go.Scatter(
    x=node_x, y=node_y, text=texts, hovertext=hovers,
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        color=node_color,
        size=40,
        line_width=2))
  
  return node_trace


def get_edges_from_graph(graph, pos, edge_color):
  edge_x = []
  edge_y = []
  hovers = []
  xtext = []
  ytext = []
  for edge in graph.edges():
    relation = graph.edges[edge]['attr_dict']['relation']
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    hovers.append(relation)
    xtext.append((x0+x1)/2)
    ytext.append((y0+y1)/2)

  edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color=edge_color),
    mode='lines')
  
  labels_trace = go.Scatter(x=xtext,y= ytext, mode='text',
                              textfont = {'color': edge_color},
                              marker_size=0.5,
                              text=hovers,
                              textposition='top center',
                              hovertemplate='weight: %{text}<extra></extra>')
  return edge_trace, labels_trace


def show_graph_in_plotly(graph, node_color='white', edge_color='grey'):
  pos = nx.spring_layout(graph)
  node_trace = get_nodes_from_graph(graph, pos, node_color)
  edge_trace, labels_trace = get_edges_from_graph(graph, pos, edge_color)
  fig = go.Figure(data=[edge_trace, node_trace, labels_trace],
               layout=go.Layout(
                  title='Company Ecosystem',
                  titlefont_size=16,                   
                  showlegend=False,
                  width=1600,
                  height=1000,
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers')) 
  fig.show()

# Relation Extraction Pipeline

### Create Generic Base Pipeline
This pipeline will:
1) Split Text into Sentences
2) Split Sentences into Words
3) Use FInancial Text Embeddings to obtain numerical semantic representation of words

In [ ]:
def get_generic_base_pipeline():
    document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

    sentence_detector = nlp.SentenceDetector()\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

    tokenizer = nlp.Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

    embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")
    
    base_pipeline = Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings
    ])
    
    return base_pipeline
    
generic_base_pipeline = get_generic_base_pipeline()

In [ ]:
import pandas as pd

def get_relations_df (results, col='relations'):
    rel_pairs=[]
    for rel in results[0][col]:
        rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

    rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

    return rel_df

# Sample Texts from Cadence Design System
Examples taken from publicly available information about Cadence in SEC's Edgar database [here](https://www.sec.gov/Archives/edgar/data/813672/000081367222000012/cdns-20220101.htm) and [Wikipedia](https://en.wikipedia.org/wiki/Cadence_Design_Systems)

## NER: Named Entity Recognition
Main component to carry out information extraction and extract entities from texts

In [ ]:
summary_sample_text = ["""
 
UNITED STATES SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
_____________________________________ 
FORM 10-K
_____________________________________  
(Mark One) 
☒	ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
 
For the fiscal year ended January 1, 2022
OR 
☐	TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
 
For the transition period from _________ to_________.

Commission file number 000-15867
_____________________________________ 
cdns-20220101_g1.jpg
CADENCE DESIGN SYSTEMS, INC.
(Exact name of registrant as specified in its charter)
____________________________________  
Delaware	 	00-0000000
(State or Other Jurisdiction of
Incorporation or Organization)	 	(I.R.S. Employer
Identification No.)
2655 Seely Avenue, Building 5,	San Jose,	California	 	95134
(Address of Principal Executive Offices)	 	(Zip Code)
 
(408)-943-1234
(Registrant’s Telephone Number, including Area Code)
Securities registered pursuant to Section 12(b) of the Act: 
Title of Each Class	Trading Symbol(s)	Names of Each Exchange on which Registered
Common Stock, $0.01 par value per share	CDNS	Nasdaq Global Select Market
 
Securities registered pursuant to Section 12(g) of the Act:
None"""]
            

In [ ]:
ner_model_sec10k = finance.NerModel.pretrained("finner_sec_10k_summary", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_summary")

ner_converter_sec10k = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_summary"])\
    .setOutputCol("ner_chunk_sec10k")

summary_pipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner_model_sec10k,
    ner_converter_sec10k
])

In [ ]:
from sparknlp_display import NerVisualizer

ner_vis = viz.NerVisualizer()

empty_data = spark.createDataFrame([[""]]).toDF("text")

summary_model = summary_pipeline.fit(empty_data)

light_summary_model = LightPipeline(summary_model)

summary_results = light_summary_model.fullAnnotate(summary_sample_text)

### Visualize Results

In [ ]:
for r in summary_results:
    displayHTML(ner_vis.display(r, label_col = "ner_chunk_sec10k", document_col = "document", return_html=True))

UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 _____________________________________ FORM 10-K _____________________________________ (Mark One) ☒	ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended January 1, 2022 FISCAL_YEAR OR ☐	TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from _________ to_________. Commission file number 000-15867 CFN _____________________________________ cdns-20220101_g1.jpg CADENCE DESIGN SYSTEMS, INC ORG . (Exact name of registrant as specified in its charter) ____________________________________ Delaware STATE 00-0000000 IRS (State or Other Jurisdiction of Incorporation or Organization) (I.R.S. Employer Identification No.) 2655 Seely Avenue, Building 5,	San Jose,	California ADDRESS 95134 (Address of Principal Executive Offices) (Zip Code) (408)-943-1234 PHONE (Registrant’s Telephone Number, including Area Code) Securities registered pursuant to Section 12(b) of the Act: Title of Each Class	Trading Symbol(s)	Names of Each Exchange on which Registered Common Stock TITLE_CLASS , $0.01 TITLE_CLASS_VALUE par value per share CDNS TICKER Nasdaq Global Select Market STOCK_EXCHANGE Securities registered pursuant to Section 12(g) of the Act: None

## First, let's extract the Organization from NER results

In [ ]:
G.clear()
G.nodes()

Out[15]: NodeView(())

In [ ]:
ORG = next(filter(lambda x: x.metadata['entity']=='ORG', summary_results[0]['ner_chunk_sec10k'])).result
ORG

Out[16]: 'CADENCE DESIGN SYSTEMS, INC'

In [ ]:
# I add our main Organization in the center (x=0, y=0)
G.add_node(ORG, attr_dict={'entity': 'ORG'})

### Let's create a node for the Company we are processing

In [ ]:
show_graph_in_plotly(G)

Then, let's add all the summary information from SEC 10K filings (1st page) to that organization.

We can create nodes and add a relation to Cadence directly, since we know it's information of that company.

In [ ]:
for i, r in enumerate(summary_results[0]['ner_chunk_sec10k']):
  text = r.result
  entity = r.metadata['entity']
  
  if entity == 'ORG':
    continue #Already added
  G.add_node(text, attr_dict={'entity': entity}),
  G.add_edge(ORG, text, attr_dict={'relation': 'has_' + entity.lower()})  

In [ ]:
show_graph_in_plotly(G)

## Normalizing the company name to query John Snow Labs datasources for more information about Cadence

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("ner_chunk") \
      .setOutputCol("sentence_embeddings")
    
resolver = finance.SentenceEntityResolverModel.pretrained("finel_edgar_company_name", "en", "finance/models")\
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("normalization")\
      .setDistanceFunction("EUCLIDEAN")

pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          embeddings,
          resolver])

lp = LightPipeline(pipelineModel)

normalized_org = lp.fullAnnotate(ORG)
normalized_org

In [ ]:
NORM_ORG = normalized_org[0]['normalization'][0].result
NORM_ORG

Out[22]: 'CADENCE DESIGN SYSTEMS INC'

### NORMALIZED NAME
In Edgar, the company official is different! We need to take it before being able to augment with external information in EDGAR.

- Incorrect: `CADENCE DESIGN SYSTEMS, INC`
- Correct (Official): `CADENCE DESIGN SYSTEM INC`

In [ ]:
  G.add_node(NORM_ORG, attr_dict={'entity': 'ORG'}),
  G.add_edge(ORG, NORM_ORG, attr_dict={'relation': 'has_official_name'})  

## DATA AUGMENTATION WITH CHUNK MAPPER

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

chunkAssembler = nlp.Doc2Chunk() \
    .setInputCols("document") \
    .setOutputCol("chunk") \
    .setIsArray(False)

CM = finance.ChunkMapperModel()\
      .pretrained("finmapper_edgar_companyname", "en", "finance/models")\
      .setInputCols(["chunk"])\
      .setOutputCol("mappings")
      
cm_pipeline = Pipeline(stages=[documentAssembler, chunkAssembler, CM])
fit_cm_pipeline = cm_pipeline.fit(empty_data)

df = spark.createDataFrame([[NORM_ORG]]).toDF("text")
r = fit_cm_pipeline.transform(df).collect()

In [ ]:
mappings = r[0]['mappings']
for mapping in mappings:
  text = mapping.result
  relation = mapping.metadata['relation']
  print(f"{ORG} - has_{relation} - {text}")
    
  G.add_node(text, attr_dict={'entity': relation}),
  G.add_edge(ORG, text, attr_dict={'relation': 'has_' + relation.lower()})  

CADENCE DESIGN SYSTEMS, INC - has_name - CADENCE DESIGN SYSTEMS INC
CADENCE DESIGN SYSTEMS, INC - has_sic - SERVICES-PREPACKAGED SOFTWARE [7372]
CADENCE DESIGN SYSTEMS, INC - has_sic_code - 7372
CADENCE DESIGN SYSTEMS, INC - has_irs_number - 770148231
CADENCE DESIGN SYSTEMS, INC - has_fiscal_year_end - 1228
CADENCE DESIGN SYSTEMS, INC - has_state_location - CA
CADENCE DESIGN SYSTEMS, INC - has_state_incorporation - DE
CADENCE DESIGN SYSTEMS, INC - has_business_street - 2655 SEELY AVENUE BLDG 5
CADENCE DESIGN SYSTEMS, INC - has_business_city - SAN JOSE
CADENCE DESIGN SYSTEMS, INC - has_business_state - CA
CADENCE DESIGN SYSTEMS, INC - has_business_zip - 95134
CADENCE DESIGN SYSTEMS, INC - has_business_phone - 4089431234
CADENCE DESIGN SYSTEMS, INC - has_former_name - ECAD INC /DE/
CADENCE DESIGN SYSTEMS, INC - has_former_name_date - 19880609
CADENCE DESIGN SYSTEMS, INC - has_date - 2017-02-10
CADENCE DESIGN SYSTEMS, INC - has_company_id - 813672

In [ ]:
show_graph_in_plotly(G)

## NER and Relation Extraction
NER only extracts isolated entities by itself. But you can combine some NER with specific Relation Extraction Annotators trained for them, to retrieve if the entities are related to each other.

### Companies Information

In [ ]:
ner_model_dates = finance.NerModel.pretrained("finner_org_per_role_date", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_date")

ner_converter_dates = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_date"])\
    .setOutputCol("ner_chunk_date")\
    .setWhiteList(["DATE"])

ner_model_alias = finance.NerModel.pretrained("finner_orgs_prods_alias","en","finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_alias")

ner_converter_alias = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_alias"])\
    .setOutputCol("ner_chunk_alias")\

chunk_merger = finance.ChunkMergeApproach()\
    .setInputCols("ner_chunk_alias", "ner_chunk_date")\
    .setOutputCol('ner_chunk')\
    .setMergeOverlapping(True)\

pos = PerceptronModel.pretrained()\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos")

dependency_parser = DependencyParserModel().pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos", "token"])\
    .setOutputCol("dependencies")

re_ner_chunk_filter_alias = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunk_alias")\
    .setRelationPairs(["ORG-ALIAS"])\
    .setMaxSyntacticDistance(5)

re_ner_chunk_filter_acq = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunk_acq")\
    .setRelationPairs(["DATE-ORG", "ORG-ORG"])\
    .setMaxSyntacticDistance(5)

re_model_acq = finance.RelationExtractionDLModel.pretrained("finre_acquisitions_subsidiaries", "en", "finance/models")\
    .setInputCols(["re_ner_chunk_acq", "sentence"])\
    .setOutputCol("relations_acq")\
    .setPredictionThreshold(0.5)

re_model_alias = finance.RelationExtractionDLModel().pretrained("finre_org_prod_alias", "en", "finance/models")\
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunk_alias", "sentence"])\
    .setOutputCol("relations_alias")

annotation_merger = finance.AnnotationMerger()\
    .setInputCols("relations_alias", "relations_acq")\
    .setInputType("ner_chunk")\
    .setOutputCol("relations")

nlpPipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner_model_dates,
    ner_converter_dates,
    ner_model_alias,
    ner_converter_alias,
    chunk_merger,
    pos,
    dependency_parser,
    re_ner_chunk_filter_alias,
    re_ner_chunk_filter_acq,
    re_model_acq,
    re_model_alias,
    annotation_merger
])


model = nlpPipeline.fit(empty_data)
light_model = LightPipeline(model)

In [ ]:
sample_text = [
  """Cadence acquired all of the outstanding equity of AWR Corporation ("AWR") on January 15, 2020. On February 6, 2020, Cadence also acquired all of the outstanding equity of Integrand Software Inc.""",
               
"""Cadence Design Systems was founded on 1988 by the merger of Solomon Design Automation and SDA"""]
            

In [ ]:
rel_df = pd.DataFrame()

for i in range(len(sample_text)):
    result = light_model.fullAnnotate(sample_text[i])
    rel_df = pd.concat([rel_df,get_relations_df(result)],axis = 0,ignore_index=True)

rel_df[rel_df["relation"] != "no_rel"]

Out[29]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_alias,ORG,50,64,AWR Corporation,ALIAS,68,70,AWR,0.9415686
1,was_acquired_by,ORG,0,6,Cadence,ORG,50,64,AWR Corporation,0.6954847
2,was_acquired,ORG,50,64,AWR Corporation,DATE,77,92,"January 15, 2020",0.94789404
3,was_acquired,DATE,98,113,"February 6, 2020",ORG,116,122,Cadence,0.91787183
4,was_acquired,DATE,98,113,"February 6, 2020",ORG,171,192,Integrand Software Inc,0.8110341
5,was_acquired_by,ORG,116,122,Cadence,ORG,171,192,Integrand Software Inc,0.8877772
6,was_acquired,ORG,0,21,Cadence Design Systems,DATE,38,41,1988,0.9816823
7,is_subsidiary_of,ORG,0,21,Cadence Design Systems,ORG,60,84,Solomon Design Automation,0.9912629
8,is_subsidiary_of,ORG,0,21,Cadence Design Systems,ORG,90,92,SDA,0.94412464
9,was_acquired,DATE,38,41,1988,ORG,60,84,Solomon Design Automation,0.6182229


### Visualize Results

In [ ]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()

for i in range(len(sample_text)):
    result = light_model.fullAnnotate(sample_text[i])
    displayHTML(re_vis.display(result = result[0], relation_col = "relations", document_col = "document", exclude_relations = ["no_rel"], return_html=True, show_relations=True))

Cadence ORG acquired all of the outstanding equity of AWR Corporation ORG (" AWR ALIAS ") on January 15, 2020 DATE . On February 6, 2020 DATE , Cadence ORG also acquired all of the outstanding equity of Integrand Software Inc ORG . <polyline fill="none" points="770.0,55.0 768.7142857142856,53.79607986468223 767.4285714285713,52.593128713376224 766.1428571428572,51.39211553009376 764.8571428571429,50.194009298846574 763.5714285714287,48.999779003646445 762.2857142857142,47.810393628505125 761.0000000000001,46.626822157434425 759.7142857142858,45.450033574446024 758.4285714285714,44.280996863551756 757.1428571428571,43.12068100876335 755.8571428571428,41.97005499409259 754.5714285714284,40.83008780355124 753.2857142857143,39.70174842115105 752.0,38.586005830903794 750.7142857142859,37.48382901682123 749.4285714285714,36.396186962915124 748.1428571428573,35.32404865319723 746.8571428571429,34.26838307167932 745.5714285714286,33.23015920237316 744.2857142857142,32.21034602929051 742.9999999999999,31.209912536443145 741.7142857142858,30.229827707842823 740.4285714285713,29.271060527501298 739.1428571428571,28.334579979430348 737.8571428571429,27.421355047641715 736.5714285714284,26.532354716147186 735.2857142857143,25.668547968958514 734.0000000000001,24.83090379008747 732.7142857142857,24.020391163545803 731.4285714285714,23.237979073345294 730.1428571428571,22.484636503497697 728.8571428571428,21.761332438014776 727.5714285714286,21.069035860908297 726.2857142857142,20.408715756190023 725.0,19.781341107871725 723.7142857142858,19.187880899965155 722.4285714285716,18.629304116482082 721.1428571428571,18.106579741434267 719.8571428571429,17.620676758833483 718.5714285714286,17.172564152691482 717.2857142857142,16.76321090702004 716.0,16.393586005830905 714.7142857142857,16.064658433135854 713.4285714285714,15.77739717294665 712.1428571428571,15.53277120927505 710.8571428571429,15.33174952613282 709.5714285714286,15.175301107531727 708.2857142857142,15.064394937483533 707.0,15.0 707.0,15.0 705.7142857142857,14.982762284422305 704.4285714285713,15.012035801409274 703.1428571428571,15.086851566949145 701.8571428571429,15.20624059703015 700.5714285714286,15.369233907640522 699.2857142857142,15.574862514768505 698.0000000000002,15.822157434402333 696.7142857142859,16.11014968253024 695.4285714285714,16.43787027514046 694.142857142857,16.80435022822123 692.8571428571429,17.208620557760792 691.5714285714284,17.64971227974738 690.2857142857143,18.126656410169232 689.0000000000001,18.638483965014576 687.7142857142859,19.184225960271654 686.4285714285716,19.762913411928704 685.1428571428572,20.37357733597396 683.8571428571429,21.01524874839565 682.5714285714286,21.686958665182026 681.2857142857141,22.387738102321304 679.9999999999999,23.116618075801746 678.7142857142858,23.87262960161157 677.4285714285713,24.654803695739016 676.1428571428572,25.462171374172325 674.8571428571429,26.293763652899724 673.5714285714286,27.148611547909454 672.2857142857143,28.02574607518976 671.0,28.92419825072886 669.7142857142858,29.842999090515008 668.4285714285713,30.78117961053642 667.1428571428571,31.737770826781354 665.8571428571428,32.71180375523804 664.5714285714286,33.7023094118947 663.2857142857142,34.70831881273959 662.0,35.728862973760926 660.7142857142858,36.762972910946964 659.4285714285714,37.809679640285935 658.1428571428571,38.868014177766064 656.8571428571429,39.93700753937559 655.5714285714286,41.015690741102766 654.2857142857142,42.103094798935814 653.0,43.19825072886297 651.7142857142857,44.300189546872474 650.4285714285714,45.40794226895255 649.1428571428571,46.52053991109146 647.8571428571429,47.63701348927743 646.5714285714286,48.75639401949867 645.2857142857142,49.877712517743454 644.0,51.0" stroke="#61AE28" stroke-width="1" /> has_alias <polyline fill="none" points="478.0,155.0 476.57142857142856,153.7960798646822 475.1428571428571,152.59312871337622 473.7142857142858,151.39211553009378 472.2857142857143,150.19400929884657 470.8571428571

Cadence Design Systems ORG was founded on 1988 DATE by the merger of Solomon Design Automation ORG and SDA ORG <polyline fill="none" points="36.0,155.0 36.765306122448976,154.33673469387753 37.53061224489796,153.6734693877551 38.29591836734694,153.01020408163268 39.06122448979592,152.3469387755102 39.826530612244895,151.68367346938777 40.59183673469387,151.0204081632653 41.35714285714286,150.35714285714292 42.12244897959183,149.69387755102042 42.88775510204082,149.03061224489798 43.65306122448979,148.36734693877554 44.418367346938766,147.70408163265307 45.18367346938775,147.0408163265306 45.94897959183673,146.3775510204082 46.714285714285715,145.71428571428572 47.4795918367347,145.05102040816328 48.244897959183675,144.38775510204084 49.01020408163265,143.7244897959184 49.775510204081634,143.0612244897959 50.54081632653062,142.3979591836735 51.30612244897958,141.734693877551 52.07142857142856,141.07142857142856 52.83673469387755,140.40816326530614 53.60204081632653,139.74489795918367 54.36734693877551,139.08163265306123 55.13265306122449,138.41836734693877 55.897959183673464,137.75510204081633 56.663265306122454,137.09183673469389 57.42857142857143,136.42857142857144 58.19387755102041,135.765306122449 58.95918367346938,135.10204081632654 59.72448979591837,134.4387755102041 60.489795918367335,133.77551020408163 61.255102040816325,133.11224489795921 62.0204081632653,132.44897959183675 62.785714285714285,131.7857142857143 63.55102040816327,131.12244897959184 64.31632653061224,130.4591836734694 65.08163265306122,129.79591836734693 65.84693877551021,129.13265306122452 66.61224489795919,128.46938775510205 67.37755102040816,127.80612244897961 68.14285714285714,127.14285714285717 68.90816326530611,126.4795918367347 69.67346938775509,125.81632653061226 70.43877551020408,125.15306122448982 71.20408163265306,124.48979591836735 71.96938775510205,123.8265306122449 72.73469387755101,123.16326530612245 73.5,122.5 73.5,122.5 74.26530612244898,121.83673469387755 75.03061224489795,121.17346938775509 75.79591836734694,120.51020408163266 76.56122448979592,119.84693877551021 77.3265306122449,119.18367346938776 78.09183673469387,118.5204081632653 78.85714285714289,117.85714285714289 79.62244897959185,117.19387755102042 80.38775510204083,116.53061224489795 81.15306122448979,115.86734693877548 81.91836734693875,115.20408163265304 82.68367346938776,114.54081632653057 83.44897959183673,113.87755102040816 84.21428571428572,113.21428571428571 84.9795918367347,112.55102040816328 85.74489795918369,111.88775510204083 86.51020408163265,111.22448979591837 87.27551020408164,110.56122448979592 88.04081632653062,109.89795918367346 88.80612244897958,109.23469387755101 89.57142857142856,108.57142857142856 90.33673469387755,107.90816326530613 91.10204081632652,107.24489795918367 91.86734693877551,106.58163265306122 92.63265306122447,105.91836734693875 93.39795918367346,105.25510204081633 94.16326530612244,104.59183673469387 94.92857142857144,103.92857142857144 95.69387755102042,103.26530612244898 96.45918367346938,102.60204081632654 97.22448979591837,101.9387755102041 97.98979591836734,101.27551020408163 98.75510204081633,100.61224489795917 99.5204081632653,99.94897959183672 100.28571428571429,99.28571428571428 101.05102040816327,98.62244897959184 101.81632653061224,97.9591836734694 102.58163265306122,97.29591836734693 103.3469387755102,96.63265306122449 104.11224489795919,95.96938775510205 104.87755102040816,95.3061224489796 105.64285714285714,94.64285714285714 106.40816326530611,93.9795918367347 107.17346938775509,93.31632653061224 107.93877551020407,92.6530612244898 108.70408163265306,91.98979591836735 109.46938775510203,91.3265306122449 110.23469387755101,90.66326530612244 111.0,90.0" stroke="#692ABE" stroke-width="1" /> is_subsidiary_of <polyline fill="none" points="435.0,55.0 431.96938775510205,53.79607986468223 428.93877551020404,52.593128713376224 425.90816326530614,51.39211553009376 422.87755102040813,50.194009298846574 419.8469387755102,48.999779003646445 

### Inserting Nodes (Tags) and Relations into a Graph
Now, with entities and Relations connecting them, we can start populating the Graph of the company.

In [ ]:
for t in rel_df.itertuples():
  relation = t.relation
  entity1 = t.entity1
  chunk1 = t.chunk1
  entity2 = t.entity2
  chunk2 = t.chunk2
  G.add_node(chunk1,  attr_dict={'entity': entity1})
  G.add_node(chunk2,  attr_dict={'entity': entity2})
  
  G.add_edge(ORG, chunk1, attr_dict={'relation': 'mentions_' + entity1.lower()})  
  G.add_edge(ORG, chunk2, attr_dict={'relation': 'mentions_' + entity2.lower()})  
  
  G.add_edge(chunk1, chunk2, attr_dict={'relation': relation.lower()})  
  

In [ ]:
show_graph_in_plotly(G)

## People's Information

In [ ]:
ner_model_role = finance.NerModel.pretrained("finner_org_per_role_date", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_role")

ner_converter_role = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_role"])\
    .setOutputCol("ner_chunk_role")

pos = PerceptronModel.pretrained()\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos")

dependency_parser = DependencyParserModel().pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos", "token"])\
    .setOutputCol("dependencies")

re_ner_chunk_filter_role = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk_role", "dependencies"])\
    .setOutputCol("re_ner_chunk_role")\
    .setRelationPairs(["PERSON-ROLE", "ORG-ROLE", "DATE-ROLE"])\
    .setMaxSyntacticDistance(5)

re_model_exp = finance.RelationExtractionDLModel.pretrained("finre_work_experience", "en", "finance/models")\
    .setInputCols(["re_ner_chunk_role", "sentence"])\
    .setOutputCol("relations")\
    .setPredictionThreshold(0.5)

nlpPipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner_model_role,
    ner_converter_role,
    pos,
    dependency_parser,
    re_ner_chunk_filter_role,
    re_model_exp,
])


model = nlpPipeline.fit(empty_data)
light_model = LightPipeline(model)

## Get Results

In [ ]:
sample_text = ["""Joseph Costello was the CEO of the company since founded in 1988 until 1997. Currently, Anirudh Devgan is the CEO since 2021"""]
            

In [ ]:
rel_df = pd.DataFrame()

for i in range(len(sample_text)):
    result = light_model.fullAnnotate(sample_text[i])
    rel_df = pd.concat([rel_df,get_relations_df(result)],axis = 0,ignore_index=True)

rel_df[rel_df["relation"] != "no_rel"]

Out[35]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_role,PERSON,0,14,Joseph Costello,ROLE,24,26,CEO,0.9403703
1,has_role_from,ROLE,24,26,CEO,DATE,60,63,1988,0.98894715
2,had_role_until,ROLE,24,26,CEO,DATE,71,74,1997,0.9704771
3,has_role,PERSON,88,101,Anirudh Devgan,ROLE,110,112,CEO,0.96097374
4,has_role_from,ROLE,110,112,CEO,DATE,120,123,2021,0.98356366


## Visualize Results

In [ ]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()

for i in range(len(sample_text)):
    result = light_model.fullAnnotate(sample_text[i])
    displayHTML(re_vis.display(result = result[0], relation_col = "relations", document_col = "document", exclude_relations = ["no_rel"], return_html=True, show_relations=True))

Joseph Costello PERSON was the CEO ROLE of the company since founded in 1988 DATE until 1997 DATE . Currently, Anirudh Devgan PERSON is the CEO ROLE since 2021 DATE <polyline fill="none" points="488.0,155.0 486.95918367346934,153.7960798646822 485.91836734693874,152.59312871337622 484.8775510204082,151.39211553009378 483.8367346938776,150.19400929884657 482.795918367347,148.99977900364644 481.7551020408163,147.81039362850512 480.7142857142859,146.62682215743445 479.6734693877551,145.45003357444605 478.6326530612246,144.28099686355176 477.5918367346938,143.12068100876337 476.55102040816325,141.97005499409258 475.51020408163254,140.83008780355124 474.469387755102,139.70174842115105 473.4285714285714,138.5860058309038 472.38775510204084,137.48382901682123 471.3469387755103,136.39618696291512 470.30612244897964,135.32404865319725 469.265306122449,134.2683830716793 468.22448979591843,133.23015920237316 467.18367346938766,132.2103460292905 466.1428571428571,131.20991253644314 465.1020408163265,130.22982770784282 464.0612244897959,129.2710605275013 463.02040816326536,128.33457997943034 461.9795918367347,127.42135504764173 460.93877551020404,126.53235471614718 459.8979591836735,125.6685479689585 458.85714285714283,124.83090379008749 457.8163265306123,124.02039116354581 456.7755102040817,123.2379790733453 455.734693877551,122.48463650349771 454.69387755102036,121.76133243801479 453.6530612244898,121.06903586090831 452.61224489795916,120.40871575619002 451.57142857142856,119.78134110787173 450.53061224489795,119.18788089996517 449.48979591836735,118.62930411648207 448.44897959183675,118.10657974143427 447.40816326530614,117.62067675883348 446.36734693877554,117.17256415269148 445.3265306122449,116.76321090702004 444.28571428571433,116.39358600583091 443.2448979591836,116.06465843313586 442.2040816326531,115.77739717294665 441.16326530612247,115.53277120927505 440.12244897959187,115.33174952613282 439.0816326530612,115.17530110753174 438.04081632653066,115.06439493748353 437.0,115.0 437.0,115.0 435.9591836734694,114.9827622844223 434.91836734693874,115.01203580140927 433.87755102040825,115.08685156694916 432.8367346938776,115.20624059703015 431.795918367347,115.36923390764053 430.75510204081627,115.57486251476851 429.71428571428584,115.82215743440234 428.6734693877551,116.11014968253025 427.6326530612246,116.43787027514047 426.59183673469386,116.80435022822121 425.5510204081632,117.20862055776078 424.5102040816326,117.64971227974738 423.46938775510205,118.12665641016923 422.4285714285714,118.63848396501459 421.3877551020409,119.18422596027166 420.34693877551024,119.7629134119287 419.30612244897964,120.37357733597398 418.26530612244903,121.01524874839565 417.2244897959184,121.68695866518203 416.18367346938766,122.3877381023213 415.14285714285705,123.11661807580174 414.10204081632656,123.87262960161156 413.0612244897959,124.654803695739 412.0204081632653,125.46217137417233 410.9795918367347,126.29376365289971 409.9387755102041,127.14861154790944 408.8979591836735,128.02574607518974 407.8571428571429,128.92419825072886 406.81632653061223,129.842999090515 405.7755102040816,130.7811796105364 404.7346938775511,131.73777082678137 403.69387755102036,132.71180375523804 402.6530612244898,133.7023094118947 401.61224489795916,134.7083188127396 400.57142857142856,135.72886297376095 399.53061224489795,136.76297291094698 398.48979591836735,137.80967964028594 397.44897959183675,138.86801417776604 396.40816326530614,139.9370075393756 395.36734693877554,141.01569074110276 394.32653061224494,142.1030947989358 393.28571428571433,143.19825072886297 392.2448979591837,144.30018954687245 391.204081632653,145.40794226895255 390.16326530612247,146.52053991109145 389.12244897959187,147.63701348927742 388.08163265306126,148.75639401949866 387.0408163265306,149.87771251774345 386.0,151.0" stroke="#128053" stroke-width="1" /> has_role_from <polyline fill="none" points="366.0,155.0 364.3163265306122,153.7960798646822 362.63265306122446,152.59312871337622 360.948979591

## Adding to graph

In [ ]:
for t in rel_df.itertuples():
  relation = t.relation
  entity1 = t.entity1
  chunk1 = t.chunk1
  entity2 = t.entity2
  chunk2 = t.chunk2
  G.add_node(chunk1,  attr_dict={'entity': entity1})
  G.add_node(chunk2,  attr_dict={'entity': entity2})
  
  G.add_edge(ORG, chunk1, attr_dict={'relation': 'mentions_' + entity1.lower()})  
  G.add_edge(ORG, chunk2, attr_dict={'relation': 'mentions_' + entity2.lower()})  
  
  G.add_edge(chunk1, chunk2, attr_dict={'relation': relation.lower()})  
  

In [ ]:
show_graph_in_plotly(G)

# Understanding the context of mentioned companies to identify COMPETITORS
Many Companies may be mentioned in the report. Most of them are just organizations in the ecosystem of the Cadence. Others, may be competitors.

We can analyze the surrounding context of the extracted `ORG` to check if they are competitors or not.

In [ ]:
ner = finance.NerModel.pretrained("finner_orgs_prods_alias", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(['ORG', 'PRODUCT'])

assertion = finance.AssertionDLModel.pretrained("finassertion_competitors", "en", "finance/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings"])\
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner,
    ner_converter,
    assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

light_model = LightPipeline(model)

### Get Results

In [ ]:
sample_text = ["""In the rapidly evolving market, certain elements of our application compete with Microsoft, Google, InFocus, Bluescape, Mersive, Barco, Nureva and Prysm. But, Oracle  and IBM are out of our league."""]

chunks=[]
entities=[]
status=[]


light_result = light_model.fullAnnotate(sample_text)[0]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

In [ ]:
df

Out[41]:

,chunks,entities,assertion
0,Microsoft,ORG,COMPETITOR
1,Google,ORG,COMPETITOR
2,InFocus,ORG,COMPETITOR
3,Bluescape,ORG,COMPETITOR
4,Mersive,ORG,COMPETITOR
5,Barco,ORG,COMPETITOR
6,Nureva,ORG,COMPETITOR
7,Prysm,ORG,COMPETITOR
8,Oracle,ORG,NO_COMPETITOR
9,IBM,ORG,NO_COMPETITOR


### Visualize Assertion Result

In [ ]:
vis = viz.AssertionVisualizer()

vis.set_label_colors({'COMPETITOR':'#008080', 'NO_COMPETITOR':'#800080'})
    
light_result = light_model.fullAnnotate(sample_text)[0]

displayHTML(vis.display(light_result, 'ner_chunk', 'assertion', return_html=True))


In the rapidly evolving market, certain elements of our application compete with Microsoft ORG COMPETITOR , Google ORG COMPETITOR , InFocus ORG COMPETITOR , Bluescape ORG COMPETITOR , Mersive ORG COMPETITOR , Barco ORG COMPETITOR , Nureva ORG COMPETITOR and Prysm ORG COMPETITOR . But, Oracle ORG NO_COMPETITOR and IBM ORG NO_COMPETITOR are out of our league.

### Adding it to the graph

In [ ]:
for t in df.itertuples():
  chunks = t.chunks
  entities = t.entities
  assertion = t.assertion

  G.add_node(chunks,  attr_dict={'entity': entities})
  
  G.add_edge(ORG, chunks, attr_dict={'relation': 'is_' + assertion.lower()})
  

In [ ]:
show_graph_in_plotly(G)

# Annex 1: Detecting Temporality and Certainty in Affirmations

In [ ]:
ner_model_role = finance.NerModel.pretrained("finner_org_per_role_date", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_role")

ner_converter_role = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_role"])\
    .setOutputCol("ner_chunk")

assertion = finance.AssertionDLModel.pretrained("finassertion_time", "en", "finance/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setMaxSentLen(1200)

assertion_pipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner_model_role,
    ner_converter_role,
    assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

assertion_model = assertion_pipeline.fit(empty_data)

light_model_assertion = LightPipeline(assertion_model)

### Get Result

In [ ]:
sample_text = ["""Joseph Costello was the CEO of the company since founded in 1988 until 1997. He was followed by Lip-Bu Tan for the 2009–2021 period. Currently, Anirudh Devgan is the CEO since 2021""",
              
              """In 2007, Cadence was rumored to be in talks with Kohlberg Kravis Roberts and Blackstone Group regarding a possible sale of the company.""",
              """In 2008, Cadence withdrew a $1.6 billion offer to purchase rival Mentor Graphics.""",
              
               """ The Cadence Giving Foundation will also support critical needs in areas such as diversity, equity and inclusion, environmental sustainability and STEM education.""",
              """This stand-alone, non-profit foundation will partner with other charitable initiatives to support critical needs in areas such as diversity, equity and inclusion, environmental sustainability and science, technology, engineering, and mathematics (“STEM”) education""",
              
              """Cadence employees could purchase common stock at a price equal to 85% of the lower of the fair market value at the beginning or the end of the applicable offering period"""]

chunks=[]               
entities=[]
status=[]

light_results = light_model_assertion.fullAnnotate(sample_text)

for light_result in light_results:
  for n,m in zip(light_result['ner_chunk'], light_result['assertion']):
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

In [ ]:
df

Out[47]:

,chunks,entities,assertion
0,Joseph Costello,PERSON,PAST
1,CEO,ROLE,PAST
2,1988,DATE,PAST
3,1997,DATE,PAST
4,2009–2021,DATE,PAST
5,Anirudh Devgan,PERSON,PRESENT
6,CEO,ROLE,PRESENT
7,2021,DATE,PRESENT
8,2007,DATE,PAST
9,Cadence,ORG,PAST


### Visualize Assertion Result

In [ ]:
vis = viz.AssertionVisualizer()

for light_result in light_results:
  displayHTML(vis.display(light_result, 'ner_chunk', 'assertion', return_html=True))


Joseph Costello PERSON PAST was the CEO ROLE PAST of the company since founded in 1988 DATE PAST until 1997 DATE PAST . He was followed by Lip-Bu Tan for the 2009–2021 DATE PAST period. Currently, Anirudh Devgan PERSON PRESENT is the CEO ROLE PRESENT since 2021 DATE PRESENT

In 2007 DATE PAST , Cadence ORG PAST was rumored to be in talks with Kohlberg Kravis Roberts and Blackstone Group ORG PAST regarding a possible sale of the company.

In 2008 DATE PAST , Cadence ORG PAST withdrew a $1.6 billion offer to purchase rival Mentor Graphics ORG PAST .

The Cadence Giving Foundation ORG FUTURE will also support critical needs in areas such as diversity, equity and inclusion, environmental sustainability and STEM education.

This stand-alone, non-profit foundation will partner with other charitable initiatives to support critical needs in areas such as diversity, equity and inclusion, environmental sustainability and science, technology, engineering, and mathematics (“STEM”) education

Cadence ORG POSSIBLE employees could purchase common stock at a price equal to 85% of the lower of the fair market value at the beginning or the end of the applicable offering period

# Annex 2: Graph Embeddings to find Node / Edge / Subgraph Similarity
When you populate several companies, you can generate Node, Edge or Subgraph Embeddings to check for node similarity and potential missed links

In [ ]:
!pip install node2vec

## Node Embeddings

In [ ]:
# pip install node2vec
from node2vec import Node2Vec

# Generate walks
node2vec = Node2Vec(G, dimensions=20, walk_length=16, num_walks=100)

In [ ]:
# Learn embeddings 
model = node2vec.fit(window=10, min_count=1)

In [ ]:
model.wv.get_vector('AWR')

Out[52]: array([-0.02652522, -0.27153963, 0.08497199, -0.19858257, -0.04536055,
 0.08251562, 0.8951953 , 0.31521553, -0.7170699 , 0.26028967,
 0.52876234, 0.11620306, 0.07119451, -0.308913 , -0.01593829,
 0.0336991 , 0.04910186, -0.2391716 , -0.29850596, -0.46639618],
 dtype=float32)

In [ ]:
for node, _ in model.wv.most_similar('AWR'):
    print(node)

January 15, 2020
AWR Corporation
Delaware
Cadence
95134
CA
January 1, 2022
1228
CDNS
Nureva

## Edge Embeddings

In [ ]:
from node2vec.edges import HadamardEmbedder
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

In [ ]:
edges_embs[('AWR', 'AWR Corporation')]

Out[55]: array([ 6.0157860e-03, -8.6146491e-03, 1.1243380e-02, 1.1806695e-01,
 1.1654351e-02, -5.8547879e-04, 6.0034513e-01, 9.8731108e-02,
 1.7592399e-01, 6.0749445e-02, 3.9436898e-01, 2.4251498e-02,
 1.5226916e-03, 1.4180225e-01, -5.5486225e-03, 6.7649889e-03,
 6.0799732e-03, 7.8225002e-02, 1.0639255e-01, 4.1596767e-01],
 dtype=float32)

In [ ]:
edges_kv = edges_embs.as_keyed_vectors()
edges_kv.most_similar(str(('AWR', 'AWR Corporation')))

Generating edge features: 0%| | 0/1326.0 [00:00<?, ?it/s]Generating edge features: 100%|██████████| 1326/1326.0 [00:00<00:00, 146432.35it/s]
Out[56]: [("('AWR Corporation', 'January 15, 2020')", 0.9908008575439453),
 ("('AWR', 'February 6, 2020')", 0.9033723473548889),
 ("('AWR Corporation', 'February 6, 2020')", 0.9018533229827881),
 ("('AWR Corporation', 'AWR Corporation')", 0.9008990526199341),
 ("('AWR Corporation', 'Cadence')", 0.9003079533576965),
 ("('January 15, 2020', 'January 15, 2020')", 0.8977513909339905),
 ("('AWR', 'Integrand Software Inc')", 0.8970919847488403),
 ("('AWR', 'January 15, 2020')", 0.8962301015853882),
 ("('AWR', 'AWR')", 0.8904980421066284),
 ("('Integrand Software Inc', 'January 15, 2020')", 0.8856679797172546)]

## Want to go further and analyse subgraph Embeddings, to compare,f or example, different Companies subgraphs?
Check `https://github.com/benedekrozemberczki/graph2vec` or get in touch with us at support@johnsnowlabs.com or in our [Slack](https://www.johnsnowlabs.com/slack-redirect/), #finance channel.